In [6]:
import numpy as np
import tensorflow as tf
import keras

SONG_DATA_DIR = "data"
SAMPLE_RATE = 48_000

INPUT_SIZE = int(100)

def build_model():
    
    #Input one second of audio data at a time at SAMPLE_RATE hz.
    input = keras.layers.Input(shape=(INPUT_SIZE, 1))
    # conv_layer1 = keras.layers.Conv1D(64, 3, activation='relu')
    # pool_layer1 = keras.layers.MaxPooling1D(2)
    # conv_layer2 = keras.layers.Conv1D(64, 3, activation='relu')
    # pool_layer2 = keras.layers.MaxPooling1D(2)
    # # flatten_1 = keras.layers.Flatten()
    # lstm_layer = keras.layers.LSTM(1024, activation='relu', return_sequences=True)
    # deconv_layer1 = keras.layers.Conv1DTranspose(64, 3, activation='relu')
    # deconv_layer2 = keras.layers.Conv1DTranspose(64, 3, activation='relu')
    # output_layer = keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same'),

    encoder = keras.models.Sequential([
        # input,
        keras.layers.Conv1D(128, 3, activation='relu'),
        keras.layers.MaxPooling1D(2),
         keras.layers.Conv1D(128, 3, activation='relu'),
        keras.layers.MaxPooling1D(2),
        # keras.layers.Conv1DTranspose(64, 3, activation='relu'),
        # keras.layers.Dense(SAMPLE_RATE)
        # conv_layer1,
        # pool_layer1,
        # conv_layer2,
        # pool_layer2,
        # flatten_1,
        # lstm_layer,
        # deconv_layer1,
        # deconv_layer2
        # output_layer
        keras.layers.Flatten(),
        # keras.layers.LSTM(1024, activation='relu'),
    ])
    
    decoder = keras.models.Sequential([
      keras.layers.Dense(INPUT_SIZE, activation='relu'),  
    ])
    
    encoded = encoder(input)
    decoded: keras.Sequential = decoder(encoded)
    
    return keras.Model(input, decoded)

model = build_model()

optimizer = keras.optimizers.Adam(learning_rate=0.00001)
loss = keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.predict(tf.zeros((1, INPUT_SIZE, 1)))
model.summary()


        

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 100, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (None, 2944)           │        49,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (None, 100)            │       294,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 344,292 (1.31 MB)

 Trainable params: 344,292 (1.31 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from scipy.io import wavfile


def take_samples(data: np.ndarray, sample_size: int):
    samples = []
    for i in range(0, len(data), sample_size):
        sample = data[i:i+sample_size]
        if len(sample) < sample_size:
            sample = np.pad(sample, (0, sample_size - len(sample)))
            
        samples.append(sample)
    
    samples = np.array(samples)
    
    return samples.reshape(samples.shape[0], samples.shape[1], 1)


def load_normalized_waveform(path: str) -> tuple[int, np.ndarray, int]:
    samplerate, data = wavfile.read(path)
    n_channels = data.shape[1]

    if n_channels > 1:
        data = data.mean(axis=1)
        data = data / np.max(data, axis=0)

    return samplerate, data


output_dir = "predictions"


sr, song_sample = load_normalized_waveform("../../data/song.wav")

samples = take_samples(song_sample, INPUT_SIZE)
x = samples
y = samples.reshape(samples.shape[0], samples.shape[1])

print(x.shape)
print(y.shape)
model.fit(x,y , epochs=20, batch_size=10)

pred = model.predict(tf.zeros((10, INPUT_SIZE, 1)))

pred = pred / np.max(pred)
pred = np.array(pred * 32767, dtype=np.int16)

pred = pred.reshape(-1)
print(pred)
wavfile.write("output.wav", 48000, pred)



(106669, 100, 1)
(106669, 100)
Epoch 1/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.3269 - loss: 0.0440
Epoch 2/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.3839 - loss: 0.0435
Epoch 3/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.4125 - loss: 0.0436
Epoch 4/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.4355 - loss: 0.0435
Epoch 5/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.4557 - loss: 0.0435
Epoch 6/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.4673 - loss: 0.0434
Epoch 7/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.4847 - loss: 0.0433
Epoch 8/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.4966 - loss: 0.0438
Epoch 9/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.5104 - loss: 0.0434
Epoch 10/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.5243 - loss: 0.0434
Epoch 11/20
10667/10667 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step 

InvalidArgumentError: Graph execution error:

Detected at node functional_11_1/sequential_6_1/flatten_1/Reshape defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\mdmoo\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\mdmoo\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\mdmoo\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\mdmoo\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Python\Lib\asyncio\base_events.py", line 608, in run_forever

  File "c:\Python\Lib\asyncio\base_events.py", line 1936, in _run_once

  File "c:\Python\Lib\asyncio\events.py", line 84, in _run

  File "C:\Users\mdmoo\AppData\Roaming\Python\Python31